# 0. Install Dependencies

In [35]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender

# 1. Test Random Environment with OpenAI Gym

In [5]:
import gym 
import random

In [6]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [7]:
actions

2

In [8]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:11.0
Episode:2 Score:22.0
Episode:3 Score:17.0
Episode:4 Score:22.0
Episode:5 Score:24.0
Episode:6 Score:16.0
Episode:7 Score:12.0
Episode:8 Score:23.0
Episode:9 Score:23.0
Episode:10 Score:44.0


# 2. Create a Deep Learning Model with Keras

In [20]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [21]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [22]:
model = build_model(states, actions)

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [24]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [25]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [26]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 7:34 - reward: 1.0000

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 73s 7ms/step - reward: 1.0000
112 episodes - episode_reward: 89.018 [9.000, 200.000] - loss: 3.190 - mae: 19.606 - mean_q: 39.693

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 75s 8ms/step - reward: 1.0000
50 episodes - episode_reward: 198.560 [180.000, 200.000] - loss: 4.796 - mae: 39.248 - mean_q: 79.326

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 75s 7ms/step - reward: 1.0000
50 episodes - episode_reward: 198.840 [182.000, 200.000] - loss: 3.963 - mae: 39.909 - mean_q: 80.215

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 76s 8ms/step - reward: 1.0000
53 episodes - episode_reward: 189.981 [144.000, 200.000] - loss: 5.040 - mae: 41.024 - mean_q: 82.349

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: 1.0000
done, took 376.001 seconds


In [27]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 198.000, steps: 198
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 188.000, steps: 188
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 182.000, steps: 182
Episode 12: reward: 195.000, steps: 195
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 187.000, steps: 187
Episode 18: reward: 183.000, steps: 183
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 188.000, steps: 188
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 196.000, steps: 196
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [28]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 170.000, steps: 170
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 173.000, steps: 173
Episode 11: reward: 196.000, steps: 196
Episode 12: reward: 177.000, steps: 177
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


# 4. Reloading Agent from Memory

In [29]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [30]:
del model
del dqn
del env

In [31]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [32]:
dqn.load_weights('dqn_weights.h5f')

In [33]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 196.000, steps: 196
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 177.000, steps: 177
